Autores: Bruno Leal Fonseca & Guilherme Namen Pimenta

Repositório Github: https://github.com/Nagi0/bh-traffic-data-mining

# Data Understanding & Data Preparation

A base de dados consiste em pastas contendo o mês e ano dos registros feitos pelos radares de Belo Horizonte. Cada mês contém subpastas para os respectivos dias, onde é possível encontrar conjuntos de arquivos no formato JSON. Nelas há registros de todos os radares. Um exemplo da estrutura de pastas é dado abaixo: 

ABRIL_2022/ 

┣ 20220401/ 

┃ ┣ 20220401_00.json 

┃ ┣ 20220401_01.json 

┃ ┣ 20220401_02.json 

┃ ┣ 20220401_03.json 

┃ ┣ 20220401_04.json 

┃ ┣ 20220401_05.json  

Nota-se que cada arquivo JSON contem os dados de uma faixa de horário no respectivo dia.

Uma das primeiras suposições é que se deve encontrar associações diretas entre as velocidades registradas com os horários, tipos de veículos e localização na via. Um determinado local pode ser mais comum encontrar velocidades mais altas em determinados horários ou em determinados tipos de veículos. Velocidades mais baixas ou altas também podem ser encontradas em horários bem comuns.Além do mais, pode-se encontrar padrões frequentes em casos que se registra infrações por excesso de velocidade.

Foram adquiridos dados do ano de 2022 inteiro, mesclando todo os JSON em dataframes. Contudo o volume de dados é muito grande, portanto, usar biblioteca como pandas não será o suficiente. A solução foi usar um framework mais adequado para Big Data, o `Polars`. Uma vez que os dado estiverem processados, transformados e as informações necessárias filtradas, eles serão levados até a modelagem.

Os atributos da base são as seguintes:
- ID EQP: Número de identificação do equipamento no sistema de 
processamento. 
- DATA HORA: Data e hora de ocorrência do evento. 
- MILESEGUNDO: Milésimo de segundo de ocorrência do evento. 
- FAIXA: Faixa de circulação da via onde ocorreu o evento. 
- ID DE ENDEREÇO: Localização do equipamento no sistema de processamento. 
- VELOCIDADE DA VIA: Velocidade regulamentada para a via. 
- VELOCIDADE AFERIDA: Velocidade medida pelo equipamento. 
- CLASSIFICACAO: Tipo de veículo registrado pelo equipamento (Automóvel/Motocicleta/Caminhão ou ônibus). 
- TAMANHO: Comprimento do veículo detectado pelo equipamento. 
- NUM SERIE: Número de identificação do equipamento. 
- LATITUDE: Localização de coordenada geográfica latitude do equipamento. 
- LONGITUDE: Localização de coordenada geográfica longitude do equipamento. 
- ENDEREÇO: Localização da via onde o equipamento se encontra instalado. 
- SENTIDO: Sentido da via fiscalizado pelo equipamento. 

As colunas que serão usadas serão as seguintes:
- "ENDEREÇO"
- "SENTIDO"
- "FAIXA"
- "CLASSIFICAÇÃO"
- "TAMANHO"
- "VELOCIDADE DA VIA"
- "VELOCIDADE AFERIDA"

Os dados contínuos serão discretizados de forma arbitrária usando visualização de histogramas. E serão gerados dois novos atributos a partir da DATA HORA, VELOCIDADE DA VIA e VELOCIDADE AFERIDA:
- "PERIODO DIA"
- "ULTRAPASSOU LIMITE"
Espera-se obter informações mais ricas usando dessa nova representação discreta dos horários, e minerar padrões em casos onde houveram infrações de trânsito.

Os códigos abaixo consistem em uma dataclass usada para modelar o banco de dados usando `Polars` dataframes, e a chamada da classe para gerar arquivos CSV que serão usados na modelagem

In [ ]:
import os
from glob import glob
from dataclasses import dataclass
from tqdm import tqdm
import polars as pl


@dataclass
class DataLoader:
    dir: str
    target: str

    def list_files(self):
        months_folders = os.listdir(self.dir)
        json_files_list = []

        for m_folder in months_folders:
            m_folder = os.path.join(self.dir, m_folder)
            if os.path.isdir(m_folder):
                d_folder_list = os.listdir(m_folder)
                for d_folder in d_folder_list:
                    d_folder = os.path.join(m_folder, d_folder)
                    json_files = glob(f"{d_folder}/*.json")
                    json_files_list.extend(json_files)

        return json_files_list

    def filter_location(self, p_df: pl.DataFrame):
        filtered_df = (
            p_df.lazy()
            .filter(pl.col("ENDEREÇO").str.contains(f"{self.target}"))
            .collect()
        )

        return filtered_df

    def check_above_speed_limite(self, p_df: pl.DataFrame):
        df = p_df.with_columns(
            pl.col("VELOCIDADE AFERIDA").cast(pl.Float32),
            pl.col("VELOCIDADE DA VIA").cast(pl.Float32),
        )
        df = df.with_columns(
            (pl.col("VELOCIDADE AFERIDA") > pl.col("VELOCIDADE DA VIA")).alias(
                "ULTRAPASSOU LIMITE"
            )
        )

        return df

    def categorize_time_of_day_extended(self, hour):
        if 0 <= hour < 6:
            return "Madrugada"
        elif 6 <= hour < 9 or 17 <= hour < 20:
            return "Pico"
        elif 9 <= hour < 12:
            return "Manhã"
        elif 12 <= hour < 17:
            return "Tarde"
        else:
            return "Noite"

    def discretize_datetime(self, p_df: pl.DataFrame):
        df = p_df.with_columns(
            pl.col("DATA HORA").str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S")
        )

        df = (
            df.lazy()
            .with_columns(
                pl.col("DATA HORA")
                .dt.hour()
                .map_elements(
                    self.categorize_time_of_day_extended, return_dtype=pl.String
                )
                .alias("PERIODO DIA")
            )
            .collect()
        )

        return df

    def load_data(self) -> pl.DataFrame:
        json_files_list = self.list_files()
        dataframes_list = []

        for file in tqdm(json_files_list):
            df = pl.read_json(file)
            filtered_df = self.filter_location(df)
            filtered_df = self.check_above_speed_limite(filtered_df)

            filtered_df = self.discretize_datetime(filtered_df)

            filtered_df = filtered_df[
                "PERIODO DIA",
                "ENDEREÇO",
                "SENTIDO",
                "FAIXA",
                "CLASSIFICAÇÃO",
                "TAMANHO",
                "VELOCIDADE DA VIA",
                "VELOCIDADE AFERIDA",
                "ULTRAPASSOU LIMITE",
            ]
            dataframes_list.append(filtered_df)

        months_df = pl.concat(dataframes_list)

        return months_df

In [ ]:
import os
from dotenv import load_dotenv
from dataLoader import DataLoader


if __name__ == "__main__":
    load_dotenv("bhTrafficDataMining/humanAttempt/dataProcessing/config/.env")
    data_loader = DataLoader("bhTrafficDataMining/data", os.environ["TARGET"])
    df = data_loader.load_data()
    df.write_csv(
        "bhTrafficDataMining/humanAttempt/dataProcessing/dataProcessed/ABRIL_2022.csv",
    )
    print(df["ENDEREÇO"].value_counts())


# Modeling

## Minerando Padrões em Casos de Infração de Trânsito

Usando dos dados salvos em CSV, pode-se filtrar apenas os casos onde houveram infrações de trânsito. O Python conta com a biblioteca `mlxtend` que permite minerar padrões frequentes com algoritmos vistos em aula e gerar regras de associação. É possível selecionar qual métrica será usada para gerar as regras de associação entre os itemsets frequentes, no caso foi usada a `confiança` com um limiar arbitrário de `75%`. A saída retornada pela biblioteca é um dataframe que contêm as regras de associação, ordenadas pelo suporte, contendo informações de várias métricas comumente usadas para avaliar regras de associação, dentre elas estão:

- support(A->C) = support(A+C) [aka 'support'], range: [0, 1]
- confidence(A->C) = support(A+C) / support(A), range: [0, 1]
- lift(A->C) = confidence(A->C) / support(C), range: [0, inf]
- leverage(A->C) = support(A->C) - support(A)*support(C), range: [-1, 1]
- conviction = [1 - support(C)] / [1 - confidence(A->C)], range: [0, inf]
- zhangs_metric(A->C) = leverage(A->C) / max(support(A->C)(1-support(A)), support(A)(support(C)-support(A->C))) range: [-1,1]

Abaixo está o código da dataclass usada para preprocessar os dados, transformando os atributos citados anteriormente em one-hot-encoders para cada classe dos atributos. Em seguida está a chamada dessa classe, levando-os para a execução dos algoritmos.

In [ ]:
from glob import glob
from dataclasses import dataclass
from tqdm import tqdm
import polars as pl


@dataclass
class DataPreprocessor:
    dir: str

    def load_database(self, p_percentage) -> pl.DataFrame:
        csv_files_list = glob(f"{self.dir}/*.csv")
        percent_index = int(round(len(csv_files_list) * p_percentage))
        dataframes_list = []
        for file in tqdm(csv_files_list[:percent_index]):
            df = pl.read_csv(file)
            df = self.discretize_speed(df)
            dataframes_list.append(df)

        return pl.concat(dataframes_list)

    def get_speep_intervals(self, p_speed):
        if 0 <= p_speed < 50:
            return "velocidadeModerada"
        elif 50 <= p_speed < 100:
            return "velocidadeAlta"
        elif p_speed >= 100:
            return "velocidadeAltissima"

    def discretize_speed(self, p_df: pl.DataFrame):
        df = (
            p_df.lazy()
            .with_columns(
                pl.col("VELOCIDADE AFERIDA")
                .map_elements(self.get_speep_intervals, return_dtype=pl.String)
                .alias("VELCOIDADE")
            )
            .collect()
        )

        return df

    def drop_undesired_columns(
        self, p_df: pl.DataFrame, p_undesired_columns: list
    ) -> pl.DataFrame:
        return p_df.drop(p_undesired_columns)

    def get_preprocessed_database(self, p_percentage: float, p_undesired_columns: list):
        df = self.load_database(p_percentage)
        df = self.drop_undesired_columns(df, p_undesired_columns)
        df = df.to_dummies()
        df = df.cast(pl.Boolean)

        return df


if __name__ == "__main__":
    data_preprocessor = DataPreprocessor(
        "bhTrafficDataMining/humanAttempt/dataProcessing/dataProcessed"
    )

    df = data_preprocessor.get_preprocessed_database(
        0.5, p_undesired_columns=["VELOCIDADE AFERIDA", "VELOCIDADE DA VIA", "TAMANHO"]
    )
    print(df)


In [ ]:
import polars as pl
from mlxtend.frequent_patterns import fpgrowth, association_rules
from preprocessor import DataPreprocessor


if __name__ == "__main__":
    data_preprocessor = DataPreprocessor(
        "bhTrafficDataMining/humanAttempt/dataProcessing/dataProcessed"
    )

    df = data_preprocessor.get_preprocessed_database(
        1.0, p_undesired_columns=["VELOCIDADE AFERIDA", "VELOCIDADE DA VIA", "TAMANHO"]
    )
    df = df.lazy().filter(pl.col("ULTRAPASSOU LIMITE_true") == True).collect()
    df = df.drop(["ULTRAPASSOU LIMITE_true", "ULTRAPASSOU LIMITE_false"])
    df = df.to_pandas()
    print(df)

    frequent_itemsets = fpgrowth(df, min_support=0.25, use_colnames=True, verbose=1)
    print(frequent_itemsets)
    frequent_itemsets.to_csv("bhTrafficDataMining/humanAttempt/resultados.csv")

    rules = association_rules(
        frequent_itemsets,
        metric="confidence",
        min_threshold=0.75,
        num_itemsets=df.shape[0],
    )
    print(rules)
    rules.to_csv("bhTrafficDataMining/humanAttempt/association_rules_conf75.csv")


### Analisando resultados

A seguir estão alguns exemplos de regras de associação com métricas de qualidade que indicam que são extremamente informativas, estabelecendo uma boa confiança, convicção, e correlações bem positivas. Em algumas delas o `coeficiente de Jaccard` é relativamente baixo, indicando um grau de pertencimento não tão interessante do antecedente e o consequente à regra.

Procurou-se regras onde tanto a medida relativa do `Lift` e a absoluta do `Levarage` indicassem uma correlação positiva entre antecedente e consequente.

Pode-se notar que algumas regras apontam que é muito comum as infrações serem registradas por alta velocidade (sendo que altíssima velocidade é apenas quando a velocidade registrada está acima de 100 km/h), sendo possível achar regras que estabelecem vínculo entre o endereço e o sentido que comumente ocorrem.

Outro aspecto notável é que a maioria das infrações se dá na faixa de alta velocidade, ou seja, a FAIXA3 que fica mais à esquerda da via.

Descobriu-se que a maioria das infrações foram cometidas por automóveis, apenas uma das regras retornadas estabelece uma associação entre motocicletas e a alta velocidade; mas como é possível notar, apesar da confiança e suporte atenderem requisitos mínimos, as demais métricas apontam que essa regra não é tão forte, mostrando que a correção entre MOTOS e ALTA VELOCIDADE é baixa

In [14]:
import pandas as pd

df = pd.read_csv("association_rules_conf75.csv").drop(columns=["Unnamed: 0"])
df = df.iloc[[2, 9, 13, 23, 36, 39, 56, 61, 64, 64, 66, 67, 89, 95]]
print("Regras de boa qualidade:")
display(df)

Regras de boa qualidade:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
2,"frozenset({'VELCOIDADE_velocidadeAlta', 'ENDER...",frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL'}),0.429657,0.566884,0.326879,0.760790,1.342055,1.0,0.083313,1.810607,0.446879,0.488125,0.447699,0.668707
9,"frozenset({'VELCOIDADE_velocidadeAlta', 'ENDER...",frozenset({'SENTIDO_Centro/Barro Preto'}),0.429657,0.445195,0.429657,1.000000,2.246206,1.0,0.238376,inf,0.972757,0.965099,1.000000,0.982550
13,"frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL', 'ENDEREÇ...",frozenset({'SENTIDO_Centro/Barro Preto'}),0.332006,0.445195,0.332006,1.000000,2.246206,1.0,0.184198,inf,0.830553,0.745754,1.000000,0.872877
23,"frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL', 'ENDEREÇ...","frozenset({'SENTIDO_Centro/Barro Preto', 'VELC...",0.332006,0.429657,0.326879,0.984558,2.291496,1.0,0.184230,36.934424,0.843726,0.751819,0.972925,0.872674
36,"frozenset({'VELCOIDADE_velocidadeAlta', 'ENDER...",frozenset({'SENTIDO_Centro/Barro Preto'}),0.318417,0.445195,0.318417,1.000000,2.246206,1.0,0.176659,inf,0.813994,0.715230,1.000000,0.857615
39,"frozenset({'ENDEREÇO_Av. do Contorno, oposto a...","frozenset({'SENTIDO_Centro/Barro Preto', 'VELC...",0.330505,0.429657,0.318417,0.963425,2.242310,1.0,0.176413,15.593785,0.827536,0.720815,0.935872,0.852260
56,"frozenset({'SENTIDO_Centro/Barro Preto', 'ENDE...",frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL'}),0.330505,0.566884,0.261191,0.790278,1.394072,1.0,0.073833,2.065186,0.422224,0.410549,0.515782,0.625513
61,"frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL', 'FAIXA_3'})","frozenset({'SENTIDO_Centro/Barro Preto', 'ENDE...",0.304165,0.445195,0.261191,0.858715,1.928850,1.0,0.125778,3.926842,0.692055,0.535041,0.745342,0.722701
64,"frozenset({'SENTIDO_Centro/Barro Preto', 'VELC...",frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL'}),0.318417,0.566884,0.256847,0.806638,1.422933,1.0,0.076342,2.239926,0.436082,0.408697,0.553557,0.629862
64,"frozenset({'SENTIDO_Centro/Barro Preto', 'VELC...",frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL'}),0.318417,0.566884,0.256847,0.806638,1.422933,1.0,0.076342,2.239926,0.436082,0.408697,0.553557,0.629862


- 2	frozenset({'VELCOIDADE_velocidadeAlta', 'ENDEREÇO_Av. do Contorno, oposto ao nº 10812'})	frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL'})	0.4296573672710922	0.5668843394993642	0.326878965006077	0.7607898523472374	1.342054806134031	1.0	0.0833129321495681	1.8106073874815145	0.4468786529788451	0.4881247598527578	0.4476991495152564	0.6687067935421507

- 9	frozenset({'VELCOIDADE_velocidadeAlta', 'ENDEREÇO_Av. do Contorno, oposto ao nº 10812'})	frozenset({'SENTIDO_Centro/Barro Preto'})	0.4296573672710922	0.4451950976190643	0.4296573672710922	1.0	2.2462062258728195	1.0	0.2383760137060881	inf	0.9727571998718928	0.9650990533564522	1.0	0.982549526678226

- 13	frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL', 'ENDEREÇO_Av. do Contorno, oposto ao nº 10812'})	frozenset({'SENTIDO_Centro/Barro Preto'})	0.3320058100731353	0.4451950976190643	0.3320058100731353	1.0	2.2462062258728195	1.0	0.1841984510475293	inf	0.8305534849662067	0.7457535176122254	1.0	0.8728767588061127

- 23	frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL', 'ENDEREÇO_Av. do Contorno, oposto ao nº 10812'})	frozenset({'SENTIDO_Centro/Barro Preto', 'VELCOIDADE_velocidadeAlta'})	0.3320058100731353	0.4296573672710922	0.326878965006077	0.9845579658201498	2.2914955981633223	1.0	0.1842302227313474	36.93442367023948	0.8437255832438068	0.7518188465220746	0.972924987027596	0.8726739090836936

- 36	frozenset({'VELCOIDADE_velocidadeAlta', 'ENDEREÇO_Av. do Contorno, oposto ao nº 10812', 'FAIXA_3'})	frozenset({'SENTIDO_Centro/Barro Preto'})	0.3184167726343447	0.4451950976190643	0.3184167726343447	1.0	2.2462062258728195	1.0	0.1766591864578502	inf	0.8139943591705997	0.7152297371135952	1.0	0.8576148685567977

- 39	frozenset({'ENDEREÇO_Av. do Contorno, oposto ao nº 10812', 'FAIXA_3'})	frozenset({'SENTIDO_Centro/Barro Preto', 'VELCOIDADE_velocidadeAlta'})	0.3305049916045497	0.4296573672710922	0.3184167726343447	0.9634250033213764	2.2423099816498744	1.0	0.1764128680715794	15.59378494932973	0.8275361625722831	0.7208148367563182	0.9358718872134386	0.852259822490683

- 56	frozenset({'SENTIDO_Centro/Barro Preto', 'ENDEREÇO_Av. do Contorno, oposto ao nº 10812', 'FAIXA_3'})	frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL'})	0.3305049916045497	0.5668843394993642	0.2611907137186576	0.7902776670652318	1.3940721448808309	1.0	0.0738326098516695	2.065186165153673	0.4222242226024719	0.4105490118669827	0.5157821522954137	0.6255127348026037

- 61	frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL', 'FAIXA_3'})	frozenset({'SENTIDO_Centro/Barro Preto', 'ENDEREÇO_Av. do Contorno, oposto ao nº 10812'})	0.3041647053864366	0.4451950976190643	0.2611907137186576	0.8587147328182565	1.9288503791050824	1.0	0.125778078011869	3.9268418671513534	0.6920552513057459	0.5350414834749698	0.7453424319514479	0.7227014700566352

- 64	frozenset({'SENTIDO_Centro/Barro Preto', 'VELCOIDADE_velocidadeAlta', 'FAIXA_3'})	frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL'})	0.3184167726343447	0.5668843394993642	0.2568472098300536	0.8066384433994788	1.4229330168334688	1.0	0.0763417279897138	2.239926426510099	0.4360820785437995	0.4086969766414953	0.5535567650058745	0.6298620911472219

- 66	frozenset({'VELCOIDADE_velocidadeAlta', 'CLASSIFICAÇÃO_AUTOMÓVEL', 'FAIXA_3'})	frozenset({'SENTIDO_Centro/Barro Preto'})	0.2976081748501816	0.4451950976190643	0.2568472098300536	0.8630381539732657	1.9385616746205343	1.0	0.1243535093753954	4.050798952232581	0.6892928051651762	0.5285399845309778	0.7531351193203865	0.7199849778957972

- 67	frozenset({'SENTIDO_Centro/Barro Preto', 'CLASSIFICAÇÃO_AUTOMÓVEL'})	frozenset({'VELCOIDADE_velocidadeAlta', 'FAIXA_3'})	0.3320058100731353	0.4038624340482931	0.2568472098300536	0.773622635620366	1.9155597807541007	1.0	0.1227625352557418	2.633379744416436	0.715514289416129	0.5361919236177441	0.6202598572726538	0.7047998061865743

- 89	frozenset({'CLASSIFICAÇÃO_MOTO'})	frozenset({'VELCOIDADE_velocidadeAlta'})	0.323767554921701	0.9211916621586494	0.3095128531182599	0.9559724203776736	1.0377562668527869	1.0	0.0112608810468963	1.7899766127817534	0.0538019080860279	0.3308718330010026	0.4413334828738753	0.6459821147506364

- 95	frozenset({'VELCOIDADE_velocidadeAlta', 'ENDEREÇO_Av. do Contorno, nº 629'})	frozenset({'SENTIDO_Praça da Estação / Rodoviária'})	0.2737531526847877	0.3136332979717435	0.2737531526847877	1.0	3.1884369627426934	1.0	0.1878950485780955	inf	0.9450873412609162	0.8728446706875216	1.0	0.9364223353437606


A seguir pode-se observar algumas regras de associação não tão interessantes, apesar de fornecerem boas métricas de qualidade a associação que revelam não é muito interessante ou não acrescenta tanta informação. Muito disso se dá pelo fato de relacionarem endereços dos semáforos com o sentido da via que estão monitorando. O que em termos estatísticos tem boa correlação, mas é uma informação pouco rica sobre o mundo real.

In [ ]:
import pandas as pd

df = pd.read_csv("association_rules_conf75.csv").drop(columns=["Unnamed: 0"])
df = df.iloc[[5, 7, 8, 30, 41, 42, 55, 62, 91, 96]]
print("Regras de boa qualidade:")
display(df)

Regras de boa qualidade:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
5,"frozenset({'ENDEREÇO_Av. do Contorno, oposto a...",frozenset({'SENTIDO_Centro/Barro Preto'}),0.445195,0.445195,0.445195,1.000000,2.246206,1.0,0.246996,inf,1.000000,1.000000,1.000000,1.000000
7,"frozenset({'SENTIDO_Centro/Barro Preto', 'VELC...","frozenset({'ENDEREÇO_Av. do Contorno, oposto a...",0.429657,0.445195,0.429657,1.000000,2.246206,1.0,0.238376,inf,0.972757,0.965099,1.000000,0.982550
8,"frozenset({'SENTIDO_Centro/Barro Preto', 'ENDE...",frozenset({'VELCOIDADE_velocidadeAlta'}),0.445195,0.921192,0.429657,0.965099,1.047664,1.0,0.019547,2.258057,0.082002,0.458678,0.557141,0.715757
30,"frozenset({'ENDEREÇO_Av. do Contorno, oposto a...",frozenset({'SENTIDO_Centro/Barro Preto'}),0.330505,0.445195,0.330505,1.000000,2.246206,1.0,0.183366,inf,0.828692,0.742382,1.000000,0.871191
41,"frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL', 'ENDEREÇ...",frozenset({'FAIXA_3'}),0.332006,0.424710,0.261191,0.786705,1.852337,1.0,0.120185,2.697161,0.688840,0.527099,0.629240,0.700846
42,"frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL', 'FAIXA_3'})","frozenset({'ENDEREÇO_Av. do Contorno, oposto a...",0.304165,0.445195,0.261191,0.858715,1.928850,1.0,0.125778,3.926842,0.692055,0.535041,0.745342,0.722701
55,"frozenset({'SENTIDO_Centro/Barro Preto', 'CLAS...","frozenset({'ENDEREÇO_Av. do Contorno, oposto a...",0.261191,0.445195,0.261191,1.000000,2.246206,1.0,0.144910,inf,0.750945,0.586688,1.000000,0.793344
56,"frozenset({'SENTIDO_Centro/Barro Preto', 'ENDE...",frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL'}),0.330505,0.566884,0.261191,0.790278,1.394072,1.0,0.073833,2.065186,0.422224,0.410549,0.515782,0.625513
57,"frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL', 'ENDEREÇ...",frozenset({'SENTIDO_Centro/Barro Preto'}),0.261191,0.445195,0.261191,1.000000,2.246206,1.0,0.144910,inf,0.750945,0.586688,1.000000,0.793344
62,"frozenset({'ENDEREÇO_Av. do Contorno, oposto a...","frozenset({'SENTIDO_Centro/Barro Preto', 'CLAS...",0.330505,0.332006,0.261191,0.790278,2.380313,1.0,0.151461,3.185136,0.866156,0.650829,0.686042,0.788491


- frozenset({'ENDEREÇO_Av. do Contorno, oposto ao nº 10812'})	frozenset({'SENTIDO_Centro/Barro Preto'})	0.4451950976190643	0.4451950976190643	0.4451950976190643	1.0	2.2462062258728195	1.0	0.2469964226750161	inf	1.0000000000000002	1.0	1.0	1.0

- frozenset({'SENTIDO_Centro/Barro Preto', 'VELCOIDADE_velocidadeAlta'})	frozenset({'ENDEREÇO_Av. do Contorno, oposto ao nº 10812'})	0.4296573672710922	0.4451950976190643	0.4296573672710922	1.0	2.2462062258728195	1.0	0.2383760137060881	inf	0.9727571998718928	0.9650990533564522	1.0	0.982549526678226

- frozenset({'SENTIDO_Centro/Barro Preto', 'ENDEREÇO_Av. do Contorno, oposto ao nº 10812'})	frozenset({'VELCOIDADE_velocidadeAlta'})	0.4451950976190643	0.9211916621586494	0.4296573672710922	0.965099053356452	1.0476636871581246	1.0	0.0195473553105041	2.2580573142109803	0.0820021906004585	0.4586782166847136	0.5571414446805465	0.7157568954163811

- frozenset({'ENDEREÇO_Av. do Contorno, oposto ao nº 10812', 'FAIXA_3'})	frozenset({'SENTIDO_Centro/Barro Preto'})	0.3305049916045497	0.4451950976190643	0.3305049916045497	1.0	2.2462062258728195	1.0	0.1833657896035741	inf	0.8286916189421824	0.7423823698241835	1.0	0.8711911849120917

- frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL', 'ENDEREÇO_Av. do Contorno, oposto ao nº 10812'})	frozenset({'FAIXA_3'})	0.3320058100731353	0.4247096719802724	0.2611907137186576	0.7867052497097015	1.8523365527362972	1.0	0.1201846350269516	2.6971612158140115	0.6888402754583336	0.5270992095842344	0.6292398117929346	0.7008458737144974

- frozenset({'CLASSIFICAÇÃO_AUTOMÓVEL', 'FAIXA_3'})	frozenset({'ENDEREÇO_Av. do Contorno, oposto ao nº 10812'})	0.3041647053864366	0.4451950976190643	0.2611907137186576	0.8587147328182565	1.9288503791050824	1.0	0.125778078011869	3.9268418671513534	0.6920552513057459	0.5350414834749698	0.7453424319514479	0.7227014700566352

- frozenset({'SENTIDO_Centro/Barro Preto', 'CLASSIFICAÇÃO_AUTOMÓVEL', 'FAIXA_3'})	frozenset({'ENDEREÇO_Av. do Contorno, oposto ao nº 10812'})	0.2611907137186576	0.4451950976190643	0.2611907137186576	1.0	2.2462062258728195	1.0	0.1449098884274867	inf	0.7509446790706189	0.5866882072950139	1.0	0.793344103647507

- frozenset({'ENDEREÇO_Av. do Contorno, oposto ao nº 10812', 'FAIXA_3'})	frozenset({'SENTIDO_Centro/Barro Preto', 'CLASSIFICAÇÃO_AUTOMÓVEL'})	0.3305049916045497	0.3320058100731353	0.2611907137186576	0.7902776670652318	2.380312762873478	1.0	0.1514611362477743	3.185136177817729	0.8661560336981305	0.6508289057918369	0.686041681054547	0.7884914583874667

- frozenset({'SENTIDO_Praça da Estação / Rodoviária'})	frozenset({'ENDEREÇO_Av. do Contorno, nº 629'})	0.3136332979717435	0.3136332979717435	0.3136332979717435	1.0	3.1884369627426934	1.0	0.215267452375111	inf	1.0000000000000002	1.0	1.0	1.0

- frozenset({'SENTIDO_Praça da Estação / Rodoviária', 'ENDEREÇO_Av. do Contorno, nº 629'})	frozenset({'VELCOIDADE_velocidadeAlta'})	0.3136332979717435	0.9211916621586494	0.2737531526847877	0.8728446706875214	0.9475169028800852	1.0	-0.0151632263821016	0.6197800616573649	-0.0746742598156805	0.2848415181508502	-0.6134755889466372	0.5850087609108874

Como visto na disciplina, uma métrica única não consegue descrever, em completude, a qualidade de uma regra de associação. É necessário, portanto, utilizar de algumas delas, cada qual fornecendo um determinado tipo de informação, cruzando-as é possível entender as características das regras e avaliar quais podem ser uteis.
Outro ponto essencial é o discernimento humano e conhecimento de um especialista, isso pode ser indispensável para certos projetos, afinal algumas regras podem retornar informação redundantes, nada surpreendentes ou mesmo incoerentes com a realidade.

Algo que vale ressaltar é o fato de que nenhuma regra estabelece qualquer relação com os horários em que as infrações ocorreram. Não pelo fato deles não serem itemsets infrequentes, mas foram eliminados pelo limiar de `confiança` mínima de 75%. Foi então gerado outro dataframe, mas dessa vez usando o `lift` como métrica e um limiar de `1.0`; isso foi feito com o objetivo de encontrar qualquer correlação minimamente surpreendente. 

O `Lift` tem a virtude de retorna se a ocorrência do antecedente o consequente juntos é maior que a ocorrência deles separadamente, ou seja uma razão entre a probabilidade conjunta deles e o produto das probabilidade individuas; se o `Lift` for menor que `1.0` então existe uma correlação negativa entre o antecedente e o consequente, se for igual a `1.0` não existe qualquer correlação, e se for bem maior que `1.0` então há uma regra interessante, pois existem uma correlação forte entre eles.

Contudo, mesmo assim, não foi retornada qualquer regra de associação para os dados de infração de trânsito que constate qualquer correlação positiva desses eventos com os horários. Pode-se concluir que, muito provavelmente, uma infração de trânsito de Belo Horizonte pode ocorrer em qualquer horário, pois não há correlações positivas entre os horários que as infrações ocorreram com outros atributos frequentes minerados, considerando um `minsup=0.25`